<a href="https://colab.research.google.com/gist/KMarkert/d9fb5074fe96e717e9aa7c2e368788cb/nwm_usgs_streamflow_plots.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# install the USGS package for acquiring gauge data
!pip install --upgrade dataretrieval --quiet

In [ ]:
import io
import json
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import requests

import dataretrieval.nwis as nwis



In [ ]:
# define the reach feature id to run the queries for
reach_id = 15039097

In [ ]:
API_KEY = 'YOUR_API_KEY'
API_URL = 'API_ENDPOINT'
ANALYSIS_ASSIM_ENDPOINT = f'{API_URL}/analysis-assim'

In [ ]:
header = {
    'x-api-key': API_KEY
}

params = {
    'comids': reach_id,
    'output_format': 'csv',
}

In [ ]:
def get_analysis_assim():
    r = requests.get(ANALYSIS_ASSIM_ENDPOINT, params=params, headers=header)
    if r.status_code == 200:
        df = pd.read_csv(io.StringIO(r.text))
    else:
        raise requests.exceptions.HTTPError(r.text)
    return df

In [ ]:
df = get_analysis_assim()
df.index = pd.to_datetime(df['time'])

In [ ]:
df.head()

In [ ]:
# specify the USGS site code for which we want data.
# this site corresponds with the NWM feature id we are using
site = '11425500'

In [ ]:
# get the daily values for the last 40 years from the site with the parameter code 00060 (discharge)
gauge = nwis.get_record(sites=site, service='dv', start='1980-01-01',parameterCd='00060')

In [ ]:
# convert the cubic feet per second to cubic meters per second
# and add a new column for cms discharge
gauge['streamflow'] = gauge['00060_Mean'] * 0.02832

In [ ]:
# select the time frame from the NWM data
gauge_sel = gauge.loc[gauge.index >= '2018-09-01']

In [ ]:
fig, ax = plt.subplots()
ax.plot(gauge_sel['streamflow'])
ax.plot(df['streamflow'])